# Import & Files

In [1]:
import sklearn.linear_model
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',None)
import re
from sklearn.cluster import KMeans 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances
import nltk
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
stop_words = stopwords.words('english')
import string
import gensim
from gensim.models import Phrases
import random
from tqdm import tqdm
from gensim.models import Word2Vec 
from gensim.utils import simple_preprocess

C:\Users\asthe\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [16]:
GD=pd.read_json('./data/Glassdoor_Data.json', compression="gzip")
linkedin=pd.read_csv('./data/LinkedIn_Skills.csv')
c_jobs=pd.read_json('./data/Clustered_Jobs_1.json')
all_courses=pd.read_csv('./data/all_courses.csv')

# Preprocessing

In [4]:
#remove missing data
c_jobs = c_jobs.replace(np.nan, '', regex=True)
all_courses=all_courses.replace(np.nan, '', regex=True)

**Glassdoor**

In [3]:
GD['ColumnA']=GD['job_description'].apply(str.lower).apply(lambda x: ' '.join(x for x in x.split() if x not in stop_words)).apply(lambda x: ' '.join([WordNetLemmatizer().lemmatize(word) for word in x.split()]))

**NYC Civil Jobs**

In [5]:
c_jobs['clean_column_A']=c_jobs['ColumnA'].apply(str.lower).apply(lambda x: ' '.join(x for x in x.split() if x not in stop_words)).apply(lambda x: ' '.join([WordNetLemmatizer().lemmatize(word) for word in x.split()]))

**Courses**

In [6]:
all_courses['Description_Set']=all_courses['skills']+" "+all_courses['summary']
all_courses['Description_Set']=all_courses['Description_Set'].apply(str.lower).apply(lambda x: ' '.join(x for x in x.split() if x not in stop_words)).apply(lambda x: ' '.join([WordNetLemmatizer().lemmatize(word) for word in x.split()]))
all_courses['Description'] = all_courses.Description_Set.tolist()
all_courses['Description_Final'] = [doc.split(' ') for doc in all_courses.Description]

# Clustering

## Setting Clusters

**NYC Civil Jobs**

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
data = c_jobs['clean_column_A']

tf_idf_vectorizor = TfidfVectorizer(stop_words = 'english',#tokenizer = tokenize_and_stem,
                             max_features = 20000,ngram_range=(1,3))
tf_idf = tf_idf_vectorizor.fit_transform(data)
tf_idf_norm = normalize(tf_idf)
tf_idf_array = tf_idf_norm.toarray()

In [8]:
from sklearn.cluster import KMeans
sklearn_pca = PCA(n_components = 5)
Y_sklearn = sklearn_pca.fit_transform(tf_idf_array)
kmeans = KMeans(n_clusters=300, max_iter=600, algorithm = 'auto')
fitted = kmeans.fit(Y_sklearn)
prediction = kmeans.predict(Y_sklearn)

In [9]:
def get_top_features_cluster(tf_idf_array, prediction, n_feats):
    labels = np.unique(prediction)
    dfs = []
    for label in labels:
        id_temp = np.where(prediction==label) # indices for each cluster
        x_means = np.mean(tf_idf_array[id_temp], axis = 0) # returns average score across cluster
        sorted_means = np.argsort(x_means)[::-1][:n_feats] # indices with top 20 scores
        features = tf_idf_vectorizor.get_feature_names()
        best_features = [(features[i], x_means[i]) for i in sorted_means]
        df3 = pd.DataFrame(best_features, columns = ['features', 'score'])
        dfs.append(df3)
    return dfs
dfs = get_top_features_cluster(tf_idf_array, prediction, 50)

**Glassdoor**

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
data = GD['ColumnA']

tf_idf_vectorizor = TfidfVectorizer(stop_words = 'english',#tokenizer = tokenize_and_stem,
                             max_features = 20000,ngram_range=(1,3))
tf_idf = tf_idf_vectorizor.fit_transform(data)
tf_idf_norm = normalize(tf_idf)
tf_idf_array = tf_idf_norm.toarray()

In [5]:
from sklearn.cluster import KMeans
sklearn_pca = PCA(n_components = 5)
Y_sklearn = sklearn_pca.fit_transform(tf_idf_array)
kmeans = KMeans(n_clusters=300, max_iter=600, algorithm = 'auto')
fitted = kmeans.fit(Y_sklearn)
prediction = kmeans.predict(Y_sklearn)

In [6]:
def get_top_features_cluster(tf_idf_array, prediction, n_feats):
    labels = np.unique(prediction)
    dfs = []
    for label in labels:
        id_temp = np.where(prediction==label) # indices for each cluster
        x_means = np.mean(tf_idf_array[id_temp], axis = 0) # returns average score across cluster
        sorted_means = np.argsort(x_means)[::-1][:n_feats] # indices with top 20 scores
        features = tf_idf_vectorizor.get_feature_names()
        best_features = [(features[i], x_means[i]) for i in sorted_means]
        df3 = pd.DataFrame(best_features, columns = ['features', 'score'])
        dfs.append(df3)
    return dfs
dfs = get_top_features_cluster(tf_idf_array, prediction, 50)

## Aggregation on Clusters

**NYC Civil Jobs**

In [10]:
FF=pd.DataFrame(dfs).reset_index()
Clusters=pd.DataFrame(prediction)
CJ=c_jobs.drop('0', axis=1)
Test=CJ.join(Clusters, how='outer')
Test['300_clusters']=Test[0]
Test2=Test.drop(0, axis=1)
jobs_with_clusters=pd.merge(Test2, FF, left_on='300_clusters', right_on='index')
    #small dataframe (below) with 300 cluster aggregation
Grouped_DF=jobs_with_clusters.groupby('300_clusters').clean_column_A.agg(lambda x: ' '.join(x))
grouped_jobs=pd.DataFrame(Grouped_DF)

**Glassdoor Jobs**

In [7]:
FF=pd.DataFrame(dfs).reset_index()
Clusters=pd.DataFrame(prediction)
Test=GD.join(Clusters, how='outer')
Test['GD_300_clusters']=Test[0]
Test2=Test.drop(0, axis=1)
GD_jobs_with_clusters=pd.merge(Test2, FF, left_on='GD_300_clusters', right_on='index')
    #small dataframe (below) with 300 cluster aggregation
Grouped_DF=GD_jobs_with_clusters.groupby('GD_300_clusters').ColumnA.agg(lambda x: ' '.join(x))
GD_grouped_jobs=pd.DataFrame(Grouped_DF)

# LinkedIn Filter

## Build Linkedin Skills List

In [8]:
     #build linkedin list
linkedlist=list(linkedin['\'05'])
skills = [str(i) for i in linkedlist]
skills = [str.lower(i) for i in skills]
skills = [x for x in skills if not (x.isdigit() or x[0] == '-' and x[1:].isdigit())] #remove integers

## Job Filtering

**NYC Civil Jobs**

In [12]:
# clean_column_A splitting into a list and making a column
grouped_jobs['split_skills'] = grouped_jobs.clean_column_A.tolist()
grouped_jobs['SKILLS_LIST'] = [doc.split(' ') for doc in grouped_jobs['split_skills']]
grouped_jobs['MATCHED_COLUMN']=np.nan


In [ ]:
skills_set= set(skills)

grouped_jobs['MATCHED_COLUMN']=grouped_jobs['SKILLS_LIST'].apply(
lambda x: list(set(x) & skills_set)
)

**Glassdoor**

In [9]:
GD_grouped_jobs['split_skills'] = GD_grouped_jobs.ColumnA.tolist()
GD_grouped_jobs['SKILLS_LIST'] = [doc.split(' ') for doc in GD_grouped_jobs['split_skills']]
GD_grouped_jobs['MATCHED_COLUMN']=np.nan

In [10]:
skills_set= set(skills)

GD_grouped_jobs['MATCHED_COLUMN']=GD_grouped_jobs['SKILLS_LIST'].apply(
lambda x: list(set(x) & skills_set)
)

## Course Filtering

In [15]:
all_courses['MATCHED_COLUMN']=np.nan

In [16]:
skills_set= set(skills)

all_courses['MATCHED_COLUMN_1']=all_courses['Description_Final'].apply(
lambda x: list(set(x) & skills_set)
)

C:\Users\asthe\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
GD_all_courses=all_courses.copy()

# Optional Quantile Check

## Setup GoogleNews Word2Vec

In [122]:
import itertools

In [126]:
model_GOOGLE = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

C:\Users\asthe\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [123]:
test_skills = grouped_jobs["MATCHED_COLUMN"] #optional Glassdoor version of this too

In [124]:
test_skills_combo_final = []
for i in test_skills:
    test_skills_combo = list(itertools.combinations(i, 2))
    test_skills_combo_final.append(test_skills_combo)

In [15]:
data_skills_f = []
for index in range(len(test_skills_combo_final)):
    for x,y in test_skills_combo_final[index]:
        try:
            test = model_GOOGLE.similarity(x,y)
        except:
            pass
        data_skills_f.append([x,y,test, 'job_cluster_'+str(index+1)])

data_skills_df = pd.DataFrame(data_skills_f)

In [ ]:
#p_75 = data_skills_f['Sim'].quantile(.95)

cutoff_points = data_skills_df.groupby('Cluster').Sim.agg(lambda x: x.quantile(.95))
cutoff_points.rename(columns={ data_skills_f.columns[1]: "cut" }, inplace = True)

In [ ]:
full_data_job_skills_v1 = pd.merge(data_skills_f, cutoff_points, on='Cluster', how='left')
full_data_job_skills_v1['Above_cut'] = full_data_job_skills_v1['Sim_x']>=full_data_job_skills_v1['Sim_y']
filter_cut = full_data_job_skills_v1['Sim_x']>=full_data_job_skills_v1['Sim_y']
reduced_full_data = full_data_job_skills_v1[filter_cut]
reduced_full_v1 = reduced_full_data['Value_1'],reduced_full_data['Value_2'],reduced_full_data['Cluster']
reduced_full_v1 = pd.DataFrame(reduced_full_v1).transpose()

# Course - Job Cluster Matching (KEYWORDS)

**Run either following model, probably not both**

## Neeraj's Method

**NYC Civil Jobs**

In [ ]:
jobs=grouped_jobs.reset_index()
jobs['job_cluster_id']=jobs['300_clusters']+1
jobs=jobs.drop('200_clusters', axis=1)

In [ ]:
final_dic = {}
for i in range(jobs.shape[0]):
   var_job_id = jobs.loc[i,'job_cluster_id']
   job_skills = jobs.loc[i,'MATCHED_COLUMN']
   df = pd.DataFrame()
   df = all_courses[['course_id', 'MATCHED_COLUMN']].copy()
   df['job_vs_course_count'] = df['MATCHED_COLUMN'].apply(lambda x: len(set(x)&set(job_skills)))
   new_df = pd.DataFrame()
   new_df = df.sort_values('job_vs_course_count',ascending = False)[['course_id', 'job_vs_course_count']].head(10).copy()
   temp_list = new_df.values.tolist()
   final_dic[var_job_id] =  [var_job_id, temp_list]
final_df = pd.DataFrame(list(final_dic.values()), columns=['job_cluster_id', 'course_id - job_vs_course_count'])
final_df.head()

**Glassdoor Jobs**

In [ ]:
jobs=GD_grouped_jobs.reset_index()
jobs['job_cluster_id']=jobs['300_clusters']+1
jobs=jobs.drop('200_clusters', axis=1)

In [ ]:
final_dic = {}
for i in range(jobs.shape[0]):
   var_job_id = jobs.loc[i,'job_cluster_id']
   job_skills = jobs.loc[i,'MATCHED_COLUMN']
   df = pd.DataFrame()
   df = all_courses[['course_id', 'MATCHED_COLUMN']].copy()
   df['job_vs_course_count'] = df['MATCHED_COLUMN'].apply(lambda x: len(set(x)&set(job_skills)))
   new_df = pd.DataFrame()
   new_df = df.sort_values('job_vs_course_count',ascending = False)[['course_id', 'job_vs_course_count']].head(10).copy()
   temp_list = new_df.values.tolist()
   final_dic[var_job_id] =  [var_job_id, temp_list]
GD_final_df = pd.DataFrame(list(final_dic.values()), columns=['job_cluster_id', 'course_id - job_vs_course_count'])
GD_final_df.head()

## Keyword Columns for each Cluster

**Matching NYC Civil Jobs**

In [18]:
for i in range(grouped_jobs.shape[0]): # range = num of clusters
    tmp_set_1 = set(grouped_jobs['MATCHED_COLUMN'][i])
    
    all_courses['job_cluster_'+str(i+1)]=all_courses['MATCHED_COLUMN'].apply(
        lambda lst: len([s for s in lst if s in tmp_set_1])
    )

**Matching Glassdoor Jobs**

In [ ]:
for i in range(GD_grouped_jobs.shape[0]): # range = num of clusters
    tmp_set_1 = set(GD_grouped_jobs['MATCHED_COLUMN'][i])
    
    GD_all_courses['job_cluster_'+str(i+1)]=GD_all_courses['MATCHED_COLUMN_1'].apply(
        lambda lst: len([s for s in lst if s in tmp_set_1])
    )

## Print to CSV

In [ ]:
all_courses.drop('Unnamed: 0', axis=1, inplace=True)
# all_courses.to_csv('course_cluster_data.csv')
# job_cluster1=jobs_with_clusters.drop(['Unnamed: 0', 0, 'clusters', 'cluster_values', 'ColumnC', 'ColumnA', 'index_x', 'index_y', 'ColumnB', 'clean_column_A'], axis=1)
# job_cluster1.to_csv('job_cluster_data.csv')

# Custom Matching for Tableau

1. Matching by top 20 for each cluster
2. Connect to cluster in jobs dataframe
3. Drop index (weird column creator in Tableau)

**Glassdoor**

In [ ]:
empty=[]

for cluster_num in range(300):
    temp=list(GD_all_courses.nlargest(20, 'job_cluster_'+str(cluster_num+1), keep='first')['job_cluster_'+str(cluster_num+1)]), list(GD_all_courses.nlargest(20, 'job_cluster_'+str(cluster_num+1), keep='first').index)
    for i in range(20):
        empty.append([temp[0][i], temp[1][i], 'cluster_'+str(cluster_num+1)])

GD_course_to_cluster=pd.DataFrame(empty)
GD_course_to_cluster.columns=['values', 'course_index', 'cluster']

**NYC Civil Jobs**

In [103]:
empty=[]

for cluster_num in range(300):
    temp=list(all_courses.nlargest(20, 'job_cluster_'+str(cluster_num+1), keep='first')['job_cluster_'+str(cluster_num+1)]), list(all_courses.nlargest(20, 'job_cluster_'+str(cluster_num+1), keep='first').index)
    for i in range(20):
        empty.append([temp[0][i], temp[1][i], 'cluster_'+str(cluster_num+1)])

course_to_cluster=pd.DataFrame(empty)
course_to_cluster.columns=['values', 'course_index', 'cluster']

## Print to CSV

In [ ]:
all_courses.drop('Unnamed: 0', axis=1, inplace=True)
# all_courses.to_csv('course_cluster_data.csv')
# job_cluster1=jobs_with_clusters.drop(['Unnamed: 0', 0, 'clusters', 'cluster_values', 'ColumnC', 'ColumnA', 'index_x', 'index_y', 'ColumnB', 'clean_column_A'], axis=1)
# job_cluster1.to_csv('job_cluster_data.csv')

In [114]:
GD_course_to_cluster.to_csv('GD_course_to_cluster.csv')
course_to_cluster.to_csv('course_to_cluster.csv')

# Dare2Vec

In [59]:
# from gensim.test.utils import common_texts
# from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
# from gensim.test.utils import common_texts
# from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
# model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

# ####

# from gensim.test.utils import get_tmpfile

# fname = get_tmpfile("my_doc2vec_model")

# model.save(fname)
# model = Doc2Vec.load(fname)  # you can continue training with the loaded model!

In [ ]:
# model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

## Print to CSV

In [ ]:
# all_courses.drop('Unnamed: 0', axis=1, inplace=True)
# # all_courses.to_csv('course_cluster_data.csv')
# # job_cluster1=jobs_with_clusters.drop(['Unnamed: 0', 0, 'clusters', 'cluster_values', 'ColumnC', 'ColumnA', 'index_x', 'index_y', 'ColumnB', 'clean_column_A'], axis=1)
# # job_cluster1.to_csv('job_cluster_data.csv')